In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

/kaggle/input/suicidal-tweet-detection-dataset/Suicide_Ideation_Dataset(Twitter-based).csv


Loading Data

In [2]:
df = pd.read_csv('/kaggle/input/suicidal-tweet-detection-dataset/Suicide_Ideation_Dataset(Twitter-based).csv')


Data Cleaning ig?

In [3]:
df.isnull().sum()

Tweet      2
Suicide    0
dtype: int64

As number of rows with empty column values is quite less, we can simply remove them

In [4]:
df = df.dropna()

In [5]:
df.isnull().sum()

Tweet      0
Suicide    0
dtype: int64

In [6]:
df['Suicide'].value_counts()

Not Suicide post           1126
Potential Suicide post      659
Name: Suicide, dtype: int64

Dealing with categorical values (here, Not Suicide & Potential Suicide)

In [7]:
from sklearn.preprocessing import LabelEncoder

lab_e = LabelEncoder()

df['Suicide'] = lab_e.fit_transform(df['Suicide'])
df.head()

,Tweet,Suicide
0,making some lunch,0
1,@Alexia You want his money.,0
2,@dizzyhrvy that crap took me forever to put to...,1
3,@jnaylor #kiwitweets Hey Jer! Since when did y...,0
4,Trying out &quot;Delicious Library 2&quot; wit...,0


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vector = TfidfVectorizer(max_features = 2000)
X = vector.fit_transform(df['Tweet'])
Y = df.Suicide
X = X.toarray()

Data Split

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,random_state = 26)

In [10]:
from sklearn.metrics import accuracy_score

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='gelu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(64, activation='gelu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=5, batch_size=32)

Y_pred = model.predict(X_test)
Y_pred = np.round(Y_pred)  # Corrected from 'Y_pred - np.round(Y_pred)'
score = accuracy_score(Y_pred, Y_test)
print(f"Score: {score:.2f}")


Epoch 1/5
42/42 [==============================] - 1s 5ms/step - loss: 0.6310 - accuracy: 0.6450
Epoch 2/5
42/42 [==============================] - 0s 5ms/step - loss: 0.3040 - accuracy: 0.9245
Epoch 3/5
42/42 [==============================] - 0s 5ms/step - loss: 0.1027 - accuracy: 0.9753
Epoch 4/5
42/42 [==============================] - 0s 6ms/step - loss: 0.0496 - accuracy: 0.9910
Epoch 5/5
14/14 [==============================] - 0s 2ms/step
Score: 0.94
